In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Age-Related_Macular_Degeneration"
cohort = "GSE67899"

# Input paths
in_trait_dir = "../../input/GEO/Age-Related_Macular_Degeneration"
in_cohort_dir = "../../input/GEO/Age-Related_Macular_Degeneration/GSE67899"

# Output paths
out_data_file = "../../output/preprocess/Age-Related_Macular_Degeneration/GSE67899.csv"
out_gene_data_file = "../../output/preprocess/Age-Related_Macular_Degeneration/gene_data/GSE67899.csv"
out_clinical_data_file = "../../output/preprocess/Age-Related_Macular_Degeneration/clinical_data/GSE67899.csv"
json_path = "../../output/preprocess/Age-Related_Macular_Degeneration/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Delay and restoration of persistent wound-induced retinal pigmented epithelial-to-mesenchymal transition by TGF-beta pathway inhibitors: Implications for age-related macular degeneration"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['donor id: hfRPE-020207-2', 'donor id: hfRPE-071709', 'donor id: hfRPE-081309', 'donor id: hfRPE-111109'], 1: ['plating density: 4,000 cells/cm2', 'plating density: 80,000 cells/cm2'], 2: ['passage number: 0', 'passage number: 5'], 3: ['culture time: 3 Days', 'culture time: 16 Days', 'culture time: 32 Days', 'culture time: 64 Days'], 4: ['cultureware: T75-Flask', 'cultureware: Micropourous Membrane', 'cultureware: 6-well Multiwell Plate'], 5: ['treatment: None', 'treatment: DMSO', 'treatment: 2 ng/ml FGF2', 'treatment: 500 nM A83-01', 'treatment: 500 nM A83-01 + 2ng FGF', 'treatment: 500 nM Thiaz

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the series title and summary, this dataset seems to focus on RPE cells and the TGF-beta pathway
# It appears to contain gene expression data related to AMD
is_gene_available = True

# 2.1 Data Availability
# After analyzing the sample characteristics dictionary, I see:
# - No direct trait classification (AMD vs control) is provided
# - No age information
# - No gender information
# The dataset appears to be about cell culture experiments rather than human subjects directly

# The treatment key (index 5) seems to contain information about various treatments 
# which could be used to infer disease vs. control conditions
trait_row = 5  # Using treatment as proxy for trait
age_row = None  # No age data available
gender_row = None  # No gender data available

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert treatment information to binary where:
    0 = control condition (None or DMSO)
    1 = treatment condition (any treatment agent)
    """
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Control conditions
    if value in ['None', 'DMSO']:
        return 0
    # Treatment conditions (any other treatment)
    else:
        return 1

# No conversion functions needed for age and gender as they're not available
convert_age = None
convert_gender = None

# 3. Save Metadata
# The trait is available (inferred from treatment data)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    import pandas as pd
    import os
    
    # Create a transposed DataFrame that geo_select_clinical_features can process
    # In this format, rows are feature types and columns are samples
    # For this dataset, we don't have sample-by-sample data, so we'll create a synthetic version
    # based on the unique values in the sample characteristics
    
    # Create a mock samples dataframe where each unique treatment gets a sample
    sample_chars_dict = {0: ['donor id: hfRPE-020207-2', 'donor id: hfRPE-071709', 'donor id: hfRPE-081309', 'donor id: hfRPE-111109'], 
                        1: ['plating density: 4,000 cells/cm2', 'plating density: 80,000 cells/cm2'], 
                        2: ['passage number: 0', 'passage number: 5'], 
                        3: ['culture time: 3 Days', 'culture time: 16 Days', 'culture time: 32 Days', 'culture time: 64 Days'], 
                        4: ['cultureware: T75-Flask', 'cultureware: Micropourous Membrane', 'cultureware: 6-well Multiwell Plate'], 
                        5: ['treatment: None', 'treatment: DMSO', 'treatment: 2 ng/ml FGF2', 'treatment: 500 nM A83-01', 'treatment: 500 nM A83-01 + 2ng FGF', 
                            'treatment: 500 nM Thiazovivin', 'treatment: 500 nM Thiazovivin + 2ng FGF', 'treatment: 200 nM LDN193189', 
                            'treatment: 200 nM LDN193189 + 2ng FGF', 'treatment: 5 mM XAV939', 'treatment: 5 mM XAV939 + 2ng FGF']}
    
    # Extract the treatments (trait values) to use as samples
    treatments = sample_chars_dict[trait_row]
    
    # Create sample columns
    sample_ids = [f"Sample_{i+1}" for i in range(len(treatments))]
    
    # Create a dataframe with feature types as rows and samples as columns
    data = {}
    for i, sample_id in enumerate(sample_ids):
        data[sample_id] = [None] * 6  # 6 feature types (0-5)
        data[sample_id][trait_row] = treatments[i]  # Only set the treatment
    
    # Create the clinical dataframe in transposed format
    clinical_data = pd.DataFrame(data)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait="Treatment",  # Using "Treatment" as the trait name
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'Sample_1': [0.0], 'Sample_2': [0.0], 'Sample_3': [1.0], 'Sample_4': [1.0], 'Sample_5': [1.0], 'Sample_6': [1.0], 'Sample_7': [1.0], 'Sample_8': [1.0], 'Sample_9': [1.0], 'Sample_10': [1.0], 'Sample_11': [1.0]}
Clinical data saved to ../../output/preprocess/Age-Related_Macular_Degeneration/clinical_data/GSE67899.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '26', '27', '28', '29', '30', '31', '32'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the provided identifiers, I can see these are numeric values like '12', '13', '14', etc.
# These are not standard human gene symbols, which typically have alphanumeric formats like "BRCA1", "TP53", etc.
# These appear to be probe IDs or some other numeric identifiers that would need to be mapped to gene symbols.
# The identifiers provided are too simple to be Entrez IDs, RefSeq IDs, or Ensembl IDs.
# They require mapping to proper gene symbols before meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

### Step 6: Gene Identifier Mapping